## Import Library and dataset

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../data/interests_without_onehot.csv")

features = df.iloc[:, :-1].values
labels = df["y"].values

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [3]:
for features, label in dataset.take(31):
    print("features :", features)
    print("Label:", label)
    print("Label dtype:", label.dtype)

features : tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 0 2], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([2 1 2 0 0 0 1 2 2 0 1 2 0 0 1], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([0 2 0 0 0 1 2 2 0 0 0 1 1 0 0], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 1 1 1 1 1 1 1 1 0 1 2 1 0 1], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([2 2 0 1 1 1 2 2 1 0 2 0 2 0 2], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
feat

## Preprocessing Dataset

In [4]:
keys = tf.constant(["IT sector", "Goverment sector", "Health sector", "Education sector", "Sports sector", "Finance sector", "Entertainment sector"])
values = tf.constant([0, 1, 2, 3, 4, 5, 6])

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys, values),
    default_value=-1
)

def preprocess(features, label):
    label_int = table.lookup(label)
    label_encoded = tf.one_hot(label_int, depth=7)
    
    return features, label_encoded

dataset = dataset.map(preprocess)

In [5]:
for features, labels in dataset.take(31):
    print("Original Features:", features)
    print("One-Hot Encoded Labels:", labels.numpy())

Original Features: tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 0 2], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([2 1 2 0 0 0 1 2 2 0 1 2 0 0 1], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([0 2 0 0 0 1 2 2 0 0 0 1 1 0 0], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 1 1 1 1 1 1 1 1 0 1 2 1 0 1], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([2 2 0 1 1 1 2 2 1 0 2 0 2 0 2], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 0 2 1 1 1 0 1 2 0 0 1 0 0 0], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([0 2 1 0 0 2 2 2 0 0 0 2 2

## Splitting Dataset

In [6]:
features = []
labels = []

for feature, label in dataset:
    features.append(feature.numpy())
    labels.append(label.numpy())

features = np.array(features)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [7]:
for features, labels in train_dataset.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

Features shape: (32, 15)
Labels shape: (32, 7)


## Creating Model

In [8]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(15,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Training Model

In [9]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100,
)

Epoch 1/100


5/5 [==============================] - 1s 85ms/step - loss: 2.0804 - accuracy: 0.1572 - val_loss: 1.9046 - val_accuracy: 0.2250
Epoch 2/100
5/5 [==============================] - 0s 13ms/step - loss: 1.9555 - accuracy: 0.2075 - val_loss: 1.8218 - val_accuracy: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 1.8337 - accuracy: 0.2516 - val_loss: 1.7578 - val_accuracy: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 9ms/step - loss: 1.8463 - accuracy: 0.2830 - val_loss: 1.7013 - val_accuracy: 0.3000
Epoch 5/100
5/5 [==============================] - 0s 6ms/step - loss: 1.7968 - accuracy: 0.3019 - val_loss: 1.6516 - val_accuracy: 0.3750
Epoch 6/100
5/5 [==============================] - 0s 9ms/step - loss: 1.6723 - accuracy: 0.3836 - val_loss: 1.5959 - val_accuracy: 0.4250
Epoch 7/100
5/5 [==============================] - 0s 10ms/step - loss: 1.6857 - accuracy: 0.3396 - val_loss: 1.5358 - val_accuracy: 0.4750
Epoch 8/100
5/5 [===================

In [12]:
hasil = model.predict([[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]])
print(f"HASIL : {hasil}")

1/1 [==============================] - 0s 66ms/step
HASIL : [[0.88352317 0.783622   0.17820597 0.42466557 0.38804847 0.05463987
  0.11957355]]


## Save Model

In [13]:
model.save(filepath="../src/model/sector_classification_model.h5")